<br>

# <font color="#76b900">Retrieval-Augmented Generation with Vector Stores</font>

<br>

This notebook will progress these ideas toward the retrieval model's intended use case and explore how to build chatbot systems that rely on *vector stores* to automatically save and retrieve information.

<br>


### **Environment Setup:**

In [ ]:
# %%capture
## ^^ Comment out if you want to see the pip install process

##Necessary for Colab
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
# %pip install -q arxiv pymupdf faiss-cpu

## If you encounter a typing-extensions issue, restart your runtime and try again
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# ChatNVIDIA.get_available_models()

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

----

<br>

## RAG for Conversation History


<br>

### **Step 1**: Getting A Conversation

Consider a conversation crafted using Llama-13B between a chat agent and a blue bear named Beras. This dialogue, dense with details and potential diversions, provides a rich dataset for our study:


In [ ]:
conversation = [  ## This conversation was generated partially by an AI system, and modified to exhibit desirable properties
    "[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the rocky mountains?",
    "[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch across North America",
    "[Beras] Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard many great things about them.",
    "[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for you!"
    "[Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.",
    "[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research online or watching documentaries about them."
    "[Beras] I live in the arctic, so I'm not used to the warm climate there. I was just curious, ya know!",
    "[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains and their significance!"
]

### **Step 2:** Constructing Our Vector Store Retriever





The following shows how you can construct a FAISS vector store and reinterpret it as a retriever using the LangChain `vectorstore` API:

In [ ]:
%%time
## ^^ This cell will be timed to see how long the conversation embedding takes
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.vectorstores import FAISS

## Streamlined from_texts FAISS vectorstore construction from text list
convstore = FAISS.from_texts(conversation, embedding=embedder)
retriever = convstore.as_retriever()

CPU times: user 63.3 ms, sys: 7.98 ms, total: 71.3 ms
Wall time: 1.16 s


The retriever can now be used like any other LangChain runnable to query the vector store for some relevant documents:

In [ ]:
pprint(retriever.invoke("What is your name?"))

[
    Document(
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    ),
    Document(
        page_content='[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for 
you![Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.'
    ),
    Document(
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    )
]

In [ ]:
pprint(retriever.invoke("Where are the Rocky Mountains?"))

[
    Document(
        page_content='[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch 
across North America'
    ),
    Document(
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    ),
    Document(
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    )
]

### **Step 3:** Incorporating Conversation Retrieval Into Our Chain

Now that we have our loaded retriever component as a chain, we can incorporate it into our existing chat system as before. Specifically, we can start with an ***always-on RAG formulation*** where:
- **A retriever is always retrieving context by default**.
- **A generator is acting on the retrieved context**.

In [ ]:
from langchain.document_transformers import LongContextReorder
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

########################################################################
## Utility Runnables/Methods
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        if preface: print(preface, end="")
        pprint(x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

## Optional; Reorders longer documents to center of output text
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

In [ ]:
context_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {question}"
    "\nAnswer the user conversationally. User is not aware of context."
)

chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'question': (lambda x:x)
    }
    | context_prompt
    # | RPrint()
    | instruct_llm
    | StrOutputParser()
)

pprint(chain.invoke("Where does Beras live?"))

Based on the context provided, Beras lives in the Arctic. It seems like the Rocky Mountains are a fascinating place
for Beras to explore, given that they live in a completely different climate.

Take a second to try out some more invocations and see how the new setup performs. Regardless of your model choice, the following questions should serve as interesting starting points.

In [ ]:
pprint(chain.invoke("Where are the Rocky Mountains?"))

Hello there! The Rocky Mountains are a stunning range of mountains that span across North America. Unfortunately, 
without more context, I can't give you a precise location beyond that. However, you can find out more by doing some
research online or watching documentaries about them. I hope you enjoy learning about the Rocky Mountains!

In [ ]:
pprint(chain.invoke("Where are the Rocky Mountains? Are they close to California?"))

Hello there! The Rocky Mountains are a stunning range of mountains that span across North America. To answer your 
question about their location, they do not directly border California. In fact, they run north to south, starting 
from western Canada and stretching all the way down through the states of Montana, Idaho, Wyoming, Colorado, and 
into New Mexico. So, while they're not close to California, they do cover a significant portion of North America! 
Would you like to know more about the Rocky Mountains?

In [ ]:
pprint(chain.invoke("How far away is Beras from the Rocky Mountains?"))

Unfortunately, the context doesn't provide information on Beras' exact location or distance from the Rocky 
Mountains. I can tell you, though, that the Rocky Mountains are a beautiful and majestic range of mountains 
stretching across North America. If you'd like, I can search for more information about their geographical location
and specific details about their range.

### **Step 4:** Automatic Conversation Storage

Now that we see how our vector store memory unit should function, we can perform one last integration to allow our conversation to add new entries to our conversation: a runnable that calls the `add_texts` method for us to update the store state.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

########################################################################
## Reset knowledge base and define what it means to add more messages.
convstore = FAISS.from_texts(conversation, embedding=embedder)

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([f"User said {d.get('input')}", f"Agent said {d.get('output')}"])
    return d.get('output')

########################################################################

# instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

chat_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {input}"
    "\nAnswer the user conversationally. Make sure the conversation flows naturally.\n"
    "[Agent]"
)


conv_chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'input': (lambda x:x)
    }
    | RunnableAssign({'output' : chat_prompt | instruct_llm | StrOutputParser()})
    | partial(save_memory_and_get_output, vstore=convstore)
)

pprint(conv_chain.invoke("I'm glad you agree! I can't wait to get some ice cream there! It's such a good food!"))
print()
pprint(conv_chain.invoke("Can you guess what my favorite food is?"))
print()
pprint(conv_chain.invoke("Actually, my favorite is honey! Not sure where you got that idea?"))
print()
pprint(conv_chain.invoke("I see! Fair enough! Do you know my favorite food now?"))

While I'm sure the Rocky Mountains would provide a magnificent backdrop for enjoying ice cream, it's worth noting 
that the Rocky Mountains are known more for their stunning landscapes, diverse wildlife, and opportunities for 
outdoor activities like hiking, camping, and skiing. Even so, I'm glad you're excited about the possibility of 
visiting this beautiful place!

Well, based on our conversation, it seems like you have quite an affection for ice cream! In fact, you've mentioned
it several times and even expressed your excitement about having some in the Rocky Mountains. So, I'd guess that 
ice cream might just be your favorite food, Beras!

Well, Beras, it seems like there's been a bit of a misunderstanding! While our conversation did highlight your love
for ice cream, especially in the context of the Rocky Mountains, it's wonderful to know that your favorite food is 
actually honey. I must say, your affection for honey is just as delightful! It's fascinating how certain foods can 
bring us so much joy. I'm sure you'd enjoy a sweet, golden drizzle of honey in the Rocky Mountains just as much as 
an ice cream.

Absolutely, Beras! I do believe I know your favorite food now. While we did chat about ice cream a lot and I may 
have gotten the impression that it's your favorite, you've delightfully clarified that it's actually honey! So yes,
I'm well aware that your favorite food is the sweet, golden goodness of honey. It's fascinating how our tastes can 
surprise us, isn't it?

----

<br>

## RAG For Document Chunk Retrieval
<br>




<br>

### **Task 1**: Loading And Chunking Your Documents


In [ ]:
import json
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import ArxivLoader

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " "],
)

## TODO: Please pick some papers and add them to the list as you'd like
## NOTE: To re-use for the final assessment, make sure at least one paper is < 1 month old
print("Loading Documents")
docs = [
    ArxivLoader(query="1706.03762").load(),  ## Attention Is All You Need Paper
    ArxivLoader(query="1810.04805").load(),  ## BERT Paper
    ArxivLoader(query="2005.11401").load(),  ## RAG Paper
    ArxivLoader(query="2205.00445").load(),  ## MRKL Paper
    ArxivLoader(query="2310.06825").load(),  ## Mistral Paper
    ArxivLoader(query="2306.05685").load(),  ## LLM-as-a-Judge
    ArxivLoader(query="2408.15997").load(),  ##
    ArxivLoader(query="2406.20080").load(),  ##
    ArxivLoader(query="2408.13155").load(),  ##
    ArxivLoader(query="2408.10958").load(),  ##
    ArxivLoader(query="2408.06876").load(),  ##

    ## Some longer papers
    # ArxivLoader(query="2210.03629").load(),  ## ReAct Paper
    # ArxivLoader(query="2112.10752").load(),  ## Latent Stable Diffusion Paper
    # ArxivLoader(query="2103.00020").load(),  ## CLIP Paper
    ## TODO: Feel free to add more
]

## Cut the paper short if references is included.
## This is a standard string in papers.
for doc in docs:
    content = json.dumps(doc[0].page_content)
    if "References" in content:
        doc[0].page_content = content[:content.index("References")]

## Split the documents and also filter out stubs (overly short chunks)
print("Chunking Documents")
docs_chunks = [text_splitter.split_documents(doc) for doc in docs]
docs_chunks = [[c for c in dchunks if len(c.page_content) > 200] for dchunks in docs_chunks]

## Make some custom Chunks to give big-picture details
doc_string = "Available Documents:"
doc_metadata = []
for chunks in docs_chunks:
    metadata = getattr(chunks[0], 'metadata', {})
    doc_string += "\n - " + metadata.get('Title')
    doc_metadata += [str(metadata)]

extra_chunks = [doc_string] + doc_metadata

## Printing out some summary information for reference
pprint(doc_string, '\n')
for i, chunks in enumerate(docs_chunks):
    print(f"Document {i}")
    print(f" - # Chunks: {len(chunks)}")
    print(f" - Metadata: ")
    pprint(chunks[0].metadata)
    print()

Loading Documents
Chunking Documents


Available Documents:
 - Attention Is All You Need
 - BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
 - Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
 - MRKL Systems: A modular, neuro-symbolic architecture that combines large language models, external knowledge 
sources and discrete reasoning
 - Mistral 7B
 - Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena
 - Mamba or Transformer for Time Series Forecasting? Mixture of Universals (MoU) Is All You Need
 - AI for Extreme Event Modeling and Understanding: Methodologies and Challenges
 - Causal machine learning for sustainable agroecosystems
 - Kilometer-Scale Convection Allowing Model Emulation using Generative Diffusion Modeling
 - Decision-Focused Learning to Predict Action Costs for Planning 

Document 0
 - # Chunks: 35
 - Metadata: 


{
    'Published': '2023-08-02',
    'Title': 'Attention Is All You Need',
    'Authors': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz 
Kaiser, Illia Polosukhin',
    'Summary': 'The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural 
networks in an encoder-decoder configuration. The best\nperforming models also connect the encoder and decoder 
through an attention\nmechanism. We propose a new simple network architecture, the Transformer, based\nsolely on 
attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation
tasks show these models to be\nsuperior in quality while being more parallelizable and requiring 
significantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014\nEnglish-to-German translation 
task, improving over the existing best results,\nincluding ensembles by over 2 BLEU. On the WMT 2014 
English-to-French\ntranslation task, our model establishes a new single-model state-of-the-art\nBLEU score of 41.8 
after training for 3.5 days on eight GPUs, a small fraction\nof the training costs of the best models from the 
literature. We show that the\nTransformer generalizes well to other tasks by applying it successfully to\nEnglish 
constituency parsing both with large and limited training data.'
}


Document 1
 - # Chunks: 45
 - Metadata: 


...

{
    'Published': '2024-08-23',
    'Title': 'Causal machine learning for sustainable agroecosystems',
    'Authors': 'Vasileios Sitokonstantinou, Emiliano Díaz Salas Porras, Jordi Cerdà Bautista, Maria Piles, Ioannis 
Athanasiadis, Hannah Kerner, Giulia Martini, Lily-belle Sweet, Ilias Tsoumas, Jakob Zscheischler, Gustau 
Camps-Valls',
    'Summary': "In a changing climate, sustainable agriculture is essential for food security\nand environmental 
health. However, it is challenging to understand the complex\ninteractions among its biophysical, social, and 
economic components. Predictive\nmachine learning (ML), with its capacity to learn from data, is leveraged 
in\nsustainable agriculture for applications like yield prediction and weather\nforecasting. Nevertheless, it 
cannot explain causal mechanisms and remains\ndescriptive rather than prescriptive. To address this gap, we propose
causal\nML, which merges ML's data processing with causality's ability to reason about\nchange. This facilitates 
quantifying intervention impacts for evidence-based\ndecision-making and enhances predictive model robustness. We 
showcase causal ML\nthrough eight diverse applications that benefit stakeholders across the\nagri-food chain, 
including farmers, policymakers, and researchers."
}


Document 9
 - # Chunks: 59
 - Metadata: 


{
    'Published': '2024-08-20',
    'Title': 'Kilometer-Scale Convection Allowing Model Emulation using Generative Diffusion Modeling',
    'Authors': 'Jaideep Pathak, Yair Cohen, Piyush Garg, Peter Harrington, Noah Brenowitz, Dale Durran, Morteza 
Mardani, Arash Vahdat, Shaoming Xu, Karthik Kashinath, Michael Pritchard',
    'Summary': "Storm-scale convection-allowing models (CAMs) are an important tool for\npredicting the evolution 
of thunderstorms and mesoscale convective systems that\nresult in damaging extreme weather. By explicitly resolving
convective dynamics\nwithin the atmosphere they afford meteorologists the nuance needed to provide\noutlook on 
hazard. Deep learning models have thus far not proven skilful at\nkm-scale atmospheric simulation, despite being 
competitive at coarser\nresolution with state-of-the-art global, medium-range weather forecasting. We\npresent a 
generative diffusion model called StormCast, which emulates the\nhigh-resolution rapid refresh (HRRR) model-NOAA's 
state-of-the-art 3km\noperational CAM. StormCast autoregressively predicts 99 state variables at km\nscale using a 
1-hour time step, with dense vertical resolution in the\natmospheric boundary layer, conditioned on 26 synoptic 
variables. We present\nevidence of successfully learnt km-scale dynamics including competitive 1-6\nhour forecast 
skill for composite radar reflectivity alongside physically\nrealistic convective cluster evolution, moist 
updrafts, and cold pool\nmorphology. StormCast predictions maintain realistic power spectra for multiple\npredicted
variables across multi-hour forecasts. Together, these results\nestablish the potential for autoregressive ML to 
emulate CAMs -- opening up new\nkm-scale frontiers for regional ML weather prediction and future climate 
hazard\ndynamical downscaling."
}


Document 10
 - # Chunks: 55
 - Metadata: 


{
    'Published': '2024-08-26',
    'Title': 'Decision-Focused Learning to Predict Action Costs for Planning',
    'Authors': 'Jayanta Mandi, Marco Foschini, Daniel Holler, Sylvie Thiebaux, Jorg Hoffmann, Tias Guns',
    'Summary': 'In many automated planning applications, action costs can be hard to specify.\nAn example is the 
time needed to travel through a certain road segment, which\ndepends on many factors, such as the current weather 
conditions. A natural way\nto address this issue is to learn to predict these parameters based on input\nfeatures 
(e.g., weather forecasts) and use the predicted action costs in\nautomated planning afterward. Decision-Focused 
Learning (DFL) has been\nsuccessful in learning to predict the parameters of combinatorial optimization\nproblems 
in a way that optimizes solution quality rather than prediction\nquality. This approach yields better results than 
treating prediction and\noptimization as separate tasks. In this paper, we investigate for the first\ntime the 
challenges of implementing DFL for automated planning in order to\nlearn to predict the action costs. There are two
main challenges to overcome:\n(1) planning systems are called during gradient descent learning, to solve\nplanning 
problems with negative action costs, which are not supported in\nplanning. We propose novel methods for gradient 
computation to avoid this\nissue. (2) DFL requires repeated planner calls during training, which can limit\nthe 
scalability of the method. We experiment with different methods\napproximating the optimal plan as well as an 
easy-to-implement caching\nmechanism to speed up the learning process. As the first work that addresses\nDFL for 
automated planning, we demonstrate that the proposed gradient\ncomputation consistently yields significantly better
plans than predictions\naimed at minimizing prediction error; and that caching can temper the\ncomputation 
requirements.'
}

<br>

### **Task 2**: Construct Your Document Vector Stores

Now that we have all of the components, we can go ahead and create indices surrounding them:

In [ ]:
%%time
print("Constructing Vector Stores")
vecstores = [FAISS.from_texts(extra_chunks, embedder)]
vecstores += [FAISS.from_documents(doc_chunks, embedder) for doc_chunks in docs_chunks]

Constructing Vector Stores
CPU times: user 1.62 s, sys: 94.8 ms, total: 1.72 s
Wall time: 1min 7s


<br>

From there, we can combine our indices into a single one using the following utility:

In [ ]:
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore

embed_dims = len(embedder.embed_query("test"))
def default_FAISS():
    '''Useful utility for making an empty FAISS vectorstore'''
    return FAISS(
        embedding_function=embedder,
        index=IndexFlatL2(embed_dims),
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        normalize_L2=False
    )

def aggregate_vstores(vectorstores):
    ## Initialize an empty FAISS Index and merge others into it
    ## We'll use default_faiss for simplicity, though it's tied to your embedder by reference
    agg_vstore = default_FAISS()
    for vstore in vectorstores:
        agg_vstore.merge_from(vstore)
    return agg_vstore

## Unintuitive optimization; merge_from seems to optimize constituent vector stores away
docstore = aggregate_vstores(vecstores)

print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")

Constructed aggregate docstore with 505 chunks


<br>

### **Task 3: ** Implement Your RAG Chain

Finally, all the puzzle pieces are in place to implement the RAG pipeline! As a review, we now have:

- A way to construct a from-scratch vector store for conversational memory (and a way to initialize an empty one with `default_FAISS()`)

- A vector store pre-loaded with useful document information from our `ArxivLoader` utility (stored in `docstore`).

With the help of a couple more utilities, you're finally ready to integrate your chain! A few additional convenience utilities are provided (`doc2str` and the now-common `RPrint`) but are optional to use. Additionally, some starter prompts and structures are also defined.

> **Given all of this:** Please implement the `retrieval_chain`.

In [ ]:
from langchain.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import gradio as gr
from functools import partial
from operator import itemgetter

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")
# instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

convstore = default_FAISS()

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')

initial_msg = (
    "Hello! I am a document chat agent here to help the user!"
    f" I have access to the following documents: {doc_string}\n\nHow can I help you?"
)

chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "
    " Conversation History Retrieval:\n{history}\n\n"
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational.)"
), ('user', '{input}')])

stream_chain = chat_prompt| RPrint() | instruct_llm | StrOutputParser()

retrieval_chain = (
    {'input' : (lambda x: x)}
    ## TODO: Make sure to retrieve history & context from convstore & docstore, respectively.
    ## HINT: Our solution uses RunnableAssign, itemgetter, long_reorder, and docs2str
    | RunnableAssign({'history' : itemgetter('input') | convstore.as_retriever() | long_reorder | docs2str})
    | RunnableAssign({'context' : itemgetter('input') | docstore.as_retriever()  | long_reorder | docs2str})
    | RPrint()
)

def chat_gen(message, history=[], return_buffer=True):
    buffer = ""
    ## First perform the retrieval based on the input message
    retrieval = retrieval_chain.invoke(message)
    line_buffer = ""

    ## Then, stream the results of the stream_chain
    for token in stream_chain.stream(retrieval):
        buffer += token
        ## If you're using standard print, keep line from getting too long
        yield buffer if return_buffer else token

    ## Lastly, save the chat exchange to the conversation memory buffer
    save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)


## Start of Agent Event Loop
test_question = "Tell me about events prediction!"  ## <- modify as desired

## Before you launch your gradio interface, make sure your thing works
for response in chat_gen(test_question, return_buffer=False):
    print(response, end='')

{
    'input': 'Tell me about events prediction!',
    'history': '',
    'context': '[Quote from AI for Extreme Event Modeling and Understanding: Methodologies and Challenges] . Here, 
for example, the vegetation\\nresponse under extreme drought conditions is modeled as a video predic-\\ntion task 
of the satellite imagery while conditioning on the climate variables.\\nAnother common approach is to directly 
estimate an indicator that defines\\nthe extreme event, e.g. flood risk maps [29] or drought indices [30]. The 
esti-\\nmated variables can cover different lead times depending on the characteristics\\nof the extreme, spanning 
from short-term prediction to seasonal prediction\\n[31]. Recently, DL-based prediction techniques have gained 
popularity for their\\nability to process large data volumes, capture complex nonlinear relationships,\\nand reduce
manual feature engineering. These benefits have led to the creation\\nof global models that generalize across 
various locations, as seen in flood [32]\\nand wildfire [33] predictions.\\nProbabilistic models, unlike 
deterministic ones, focus on predicting the\\nprobability distribution of future variable states\n[Quote from AI 
for Extreme Event Modeling and Understanding: Methodologies and Challenges] . We provide an overview of how AI can 
help identify and\\nexplain extreme events more effectively, improving disaster response and\\ncommunication. We 
emphasize the need for collaboration across different\\nfields to create AI solutions that are practical, 
understandable, and trust-\\nworthy for analyzing and predicting extreme events. Such collaborative\\nefforts aim 
to enhance disaster readiness and disaster risk reduction.\\nKeywords: Artificial intelligence, extreme events, 
droughts, heatwaves,\\nfloods, detection, forecasting, modeling, understanding, attribution,\\nexplainable AI, 
causal inference, communication of risk\\nContents\\n1\\nIntroduction\\n3\\n2\\nReview of AI 
methods\\n4\\n2.1\\nExtreme Event Modeling . . . . . . . . . . . . . . . . . . . . . .\\n4\\n2.1.1\\nDetection . . 
. . . . . . . . . . . . . . . . . . . . . . . .\\n5\\n2.1.2\\nPrediction . . . . . . . . . . . . . . . . . . . . . 
. . . . .\\n6\\n2.1.3\\nImpact Assessment . . . . . . . . . . . . . . . . . . . . .\\n6\\n2\n[Quote from AI for 
Extreme Event Modeling and Understanding: Methodologies and Challenges] . 2 Components in an AI pipeline for 
extreme events. AI mainly exploits spatio-\\ntemporal Earth observation, reanalysis, and climate data to answer 
\\u201cwhat\\u2019-questions (top\\nrow): detection of events, prediction, and impact assessment. AI can also be 
used for under-\\nstanding events and thus answer \\u201cwhat if,\\u201d \\u201cwhy,\\u201d and \\u201chow 
sure\\u201d questions (middle row)\\nand makes use of explainable AI (XAI) to identify relevant drivers of events, 
causality to\\nunderstanding the system, estimate causal effects and impacts, and imagine counterfac-\\ntual 
scenarios for attribution and uncertainty estimation to quantify trust and robustness\\nfor decision-making. 
Communicating extreme events and their impacts can benefit from\\nstatistical/machine learning (bottom row) by 
improving operationalization, ensuring fair\\nand equitable narratives, and integrating language models in 
situation rooms for enhanced\\ndecision-making\n[Quote from AI for Extreme Event Modeling and Understanding: 
Methodologies and Challenges] .\\nAlternative probabilistic (ML) models have relied on surrogate 
meteorological\\nand hydrological seasonal re-forecasts [24], Gaussian processes and nonlinear\\ndependence 
measures [25] and multivariate Gaussianization [26].\\n2.1.2 Prediction\\nDesigning predictive systems that 
accurately model extreme events is essen-\\ntial to anticipating the effects of future extreme events and providing
critical\\ninformation for decision-makers to prevent damages. Spatial and temporal\\npredictions aim to provide a 
quantitative estimate of 

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: Tell me about events prediction!\n\n From this, we have retrieved the following potentially-useful 
info:  Conversation History Retrieval:\n\n\n Document Retrieval:\n[Quote from AI for Extreme Event Modeling and 
Understanding: Methodologies and Challenges] . Here, for example, the vegetation\\nresponse under extreme drought 
conditions is modeled as a video predic-\\ntion task of the satellite imagery while conditioning on the climate 
variables.\\nAnother common approach is to directly estimate an indicator that defines\\nthe extreme event, e.g. 
flood risk maps [29] or drought indices [30]. The esti-\\nmated variables can cover different lead times depending 
on the characteristics\\nof the extreme, spanning from short-term prediction to seasonal prediction\\n[31]. 
Recently, DL-based prediction techniques have gained popularity for their\\nability to process large data volumes, 
capture complex nonlinear relationships,\\nand reduce manual feature engineering. These benefits have led to the 
creation\\nof global models that generalize across various locations, as seen in flood [32]\\nand wildfire [33] 
predictions.\\nProbabilistic models, unlike deterministic ones, focus on predicting the\\nprobability distribution 
of future variable states\n[Quote from AI for Extreme Event Modeling and Understanding: Methodologies and 
Challenges] . We provide an overview of how AI can help identify and\\nexplain extreme events more effectively, 
improving disaster response and\\ncommunication. We emphasize the need for collaboration across different\\nfields 
to create AI solutions that are practical, understandable, and trust-\\nworthy for analyzing and predicting extreme
events. Such collaborative\\nefforts aim to enhance disaster readiness and disaster risk reduction.\\nKeywords: 
Artificial intelligence, extreme events, droughts, heatwaves,\\nfloods, detection, forecasting, modeling, 
understanding, attribution,\\nexplainable AI, causal inference, communication of 
risk\\nContents\\n1\\nIntroduction\\n3\\n2\\nReview of AI methods\\n4\\n2.1\\nExtreme Event Modeling . . . . . . . 
. . . . . . . . . . . . . . .\\n4\\n2.1.1\\nDetection . . . . . . . . . . . . . . . . . . . . . . . . . 
.\\n5\\n2.1.2\\nPrediction . . . . . . . . . . . . . . . . . . . . . . . . . .\\n6\\n2.1.3\\nImpact Assessment . . 
. . . . . . . . . . . . . . . . . . .\\n6\\n2\n[Quote from AI for Extreme Event Modeling and Understanding: 
Methodologies and Challenges] . 2 Components in an AI pipeline for extreme events. AI mainly exploits 
spatio-\\ntemporal Earth observation, reanalysis, and climate data to answer \\u201cwhat\\u2019-questions 
(top\\nrow): detection of events, prediction, and impact assessment. AI can also be used for under-\\nstanding 
events and thus answer \\u201cwhat if,\\u201d \\u201cwhy,\\u201d and \\u201chow sure\\u201d questions (middle 
row)\\nand makes use of explainable AI (XAI) to identify relevant drivers of events, causality to\\nunderstanding 
the system, estimate causal effects and impacts, and imagine counterfac-\\ntual scenarios for attribution and 
uncertainty estimation to quantify trust and robustness\\nfor decision-making. Communicating extreme events and 
their impacts can benefit from\\nstatistical/machine learning (bottom row) by improving operationalization, 
ensuring fair\\nand equitable narratives, and integrating language models in situation rooms for 
enhanced\\ndecision-making\n[Quote from AI for Extreme Event Modeling and Understanding: Methodologies and 
Challenges] .\\nAlternative probabilistic (ML) models have relied on surrogate meteorological\\nand hydrological 
seasonal re-forecasts [24], Gaussian processes and nonlinear\\ndependence measures [25] and multivariate 
Gaussianization [26].\\n2.1.2 Prediction\\nDesigning predictive systems

Sure, I'd be happy to help you understand event prediction, particularly in the context of extreme events.

Event prediction involves using various techniques to estimate the likelihood and potential impact of future events. In the context of extreme events, such as droughts, heatwaves, floods, and wildfires, event prediction is crucial for improving disaster response and communication.

One approach to extreme event prediction is to model the event as a video prediction task, using satellite imagery and climate variables. For example, the vegetation response under extreme drought conditions can be modeled in this way [1]. Another common approach is to estimate an indicator that defines the extreme event, such as flood risk maps or drought indices [2]. The estimated variables can cover different lead times depending on the characteristics of the extreme event, ranging from short-term prediction to seasonal prediction [3].

Recently, deep learning (DL)-based prediction techniques have g

### **Task 4:** Interact With Your Gradio Chatbot

In [ ]:
chatbot = gr.Chatbot(value = [[None, initial_msg]])
demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue()

try:
     demo.launch(debug=True, share=True, show_api=False)
     demo.close()
except Exception as e:
     demo.close()
     print(e)
     raise e

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://a791f72e9b06fff7a1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


{
    'input': 'tell me something about prediction',
    'history': "[Quote from Document] User previously responded with Tell me about events prediction!\n[Quote from 
Document] Agent previously responded with Sure, I'd be happy to help you understand event prediction, particularly 
in the context of extreme events.\n\nEvent prediction involves using various techniques to estimate the likelihood 
and potential impact of future events. In the context of extreme events, such as droughts, heatwaves, floods, and 
wildfires, event prediction is crucial for improving disaster response and communication.\n\nOne approach to 
extreme event prediction is to model the event as a video prediction task, using satellite imagery and climate 
variables. For example, the vegetation response under extreme drought conditions can be modeled in this way [1]. 
Another common approach is to estimate an indicator that defines the extreme event, such as flood risk maps or 
drought indices [2]. The estimated variables can cover different lead times depending on the characteristics of the
extreme event, ranging from short-term prediction to seasonal prediction [3].\n\nRecently, deep learning (DL)-based
prediction techniques have gained popularity for their ability to process large data volumes, capture complex 
nonlinear relationships, and reduce manual feature engineering. These benefits have led to the creation of global 
models that generalize across various locations, as seen in flood and wildfire predictions [4][5].\n\nAlternative 
probabilistic models, such as surrogate meteorological and hydrological seasonal re-forecasts, Gaussian processes, 
and nonlinear dependence measures, have also been proposed for extreme event prediction [6]. These models focus on 
predicting the probability distribution of future variable states, rather than providing a single deterministic 
prediction.\n\nOverall, AI has the potential to significantly improve extreme event prediction and understanding, 
but collaboration across different fields is essential to create practical, understandable, and trustworthy AI 
solutions for analyzing and predicting extreme events.\n\nReferences:\n[1] AI for Extreme Event Modeling and 
Understanding: Methodologies and Challenges, page 4\n[2] AI for Extreme Event Modeling and Understanding: 
Methodologies and Challenges, page 4\n[3] AI for Extreme Event Modeling and Understanding: Methodologies and 
Challenges, page 4\n[4] AI for Extreme Event Modeling and Understanding: Methodologies and Challenges, page 5\n[5] 
AI for Extreme Event Modeling and Understanding: Methodologies and Challenges, page 5\n[6] AI for Extreme Event 
Modeling and Understanding: Methodologies and Challenges, page 6\n",
    'context': '[Quote from Decision-Focused Learning to Predict Action Costs for Planning] . In this case, 
predicting the costs using ML methods is a natural choice. When the\\nground truth action costs are unknown, we 
employ a trained ML model M\\u03c9 to predict the action\\ncosts from features X. The trainable parameters, denoted
as \\u03c9, are estimated using a set of past\\nobservations, used as a training dataset for the ML model. To 
obtain a feasible plan in this setting,\\nthe action costs are first predicted using ML, followed by the generation
of a plan optimized with\\nrespect to the predicted costs. This pipeline is commonly referred to as the 
Predict-then-Optimize\\nproblem formulation in the literature [Elmachtoub and Grigas, 2022, Mandi et al., 2020]. We
present a\\nschematic diagram illustrating Predict-then-Optimize in the context of planning problems in Figure 
1.\\nVector representation.\\nState-of-the-art ML architectures, including neural networks, represent the\\ndata in
matrix and vector form\n[Quote from AI for Extreme Event Modeling and Understanding: Methodologies and Challenges] 
. Here, for example, the vegetation\\nresponse under extreme drought conditions is modeled as a video 
predic-\\ntion task of the satell

ChatPromptValue(
    messages=[
        SystemMessage(
            content="You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: tell me something about prediction\n\n From this, we have retrieved the following potentially-useful 
info:  Conversation History Retrieval:\n[Quote from Document] User previously responded with Tell me about events 
prediction!\n[Quote from Document] Agent previously responded with Sure, I'd be happy to help you understand event 
prediction, particularly in the context of extreme events.\n\nEvent prediction involves using various techniques to
estimate the likelihood and potential impact of future events. In the context of extreme events, such as droughts, 
heatwaves, floods, and wildfires, event prediction is crucial for improving disaster response and 
communication.\n\nOne approach to extreme event prediction is to model the event as a video prediction task, using 
satellite imagery and climate variables. For example, the vegetation response under extreme drought conditions can 
be modeled in this way [1]. Another common approach is to estimate an indicator that defines the extreme event, 
such as flood risk maps or drought indices [2]. The estimated variables can cover different lead times depending on
the characteristics of the extreme event, ranging from short-term prediction to seasonal prediction 
[3].\n\nRecently, deep learning (DL)-based prediction techniques have gained popularity for their ability to 
process large data volumes, capture complex nonlinear relationships, and reduce manual feature engineering. These 
benefits have led to the creation of global models that generalize across various locations, as seen in flood and 
wildfire predictions [4][5].\n\nAlternative probabilistic models, such as surrogate meteorological and hydrological
seasonal re-forecasts, Gaussian processes, and nonlinear dependence measures, have also been proposed for extreme 
event prediction [6]. These models focus on predicting the probability distribution of future variable states, 
rather than providing a single deterministic prediction.\n\nOverall, AI has the potential to significantly improve 
extreme event prediction and understanding, but collaboration across different fields is essential to create 
practical, understandable, and trustworthy AI solutions for analyzing and predicting extreme 
events.\n\nReferences:\n[1] AI for Extreme Event Modeling and Understanding: Methodologies and Challenges, page 
4\n[2] AI for Extreme Event Modeling and Understanding: Methodologies and Challenges, page 4\n[3] AI for Extreme 
Event Modeling and Understanding: Methodologies and Challenges, page 4\n[4] AI for Extreme Event Modeling and 
Understanding: Methodologies and Challenges, page 5\n[5] AI for Extreme Event Modeling and Understanding: 
Methodologies and Challenges, page 5\n[6] AI for Extreme Event Modeling and Understanding: Methodologies and 
Challenges, page 6\n\n\n Document Retrieval:\n[Quote from Decision-Focused Learning to Predict Action Costs for 
Planning] . In this case, predicting the costs using ML methods is a natural choice. When the\\nground truth action
costs are unknown, we employ a trained ML model M\\u03c9 to predict the action\\ncosts from features X. The 
trainable parameters, denoted as \\u03c9, are estimated using a set of past\\nobservations, used as a training 
dataset for the ML model. To obtain a feasible plan in this setting,\\nthe action costs are first predicted using 
ML, followed by the generation of a plan optimized with\\nrespect to the predicted costs. This pipeline is commonly
referred to as the Predict-then-Optimize\\nproblem formulation in the literature [Elmachtoub and Grigas, 2022, 
Mandi et al., 2020]. We present a\\nschematic diagram illustrating Predict-then-Optimize in the context of planning
problems in Figure 1.\\nVector representation.\\nState-of-the-art ML architectures, including neural networks, 
represent th

{
    'input': 'which are is not still being studied to have a great weather forecasting',
    'history': "[Quote from Document] Agent previously responded with Sure, I'd be happy to help you understand 
event prediction, particularly in the context of extreme events.\n\nEvent prediction involves using various 
techniques to estimate the likelihood and potential impact of future events. In the context of extreme events, such
as droughts, heatwaves, floods, and wildfires, event prediction is crucial for improving disaster response and 
communication.\n\nOne approach to extreme event prediction is to model the event as a video prediction task, using 
satellite imagery and climate variables. For example, the vegetation response under extreme drought conditions can 
be modeled in this way [1]. Another common approach is to estimate an indicator that defines the extreme event, 
such as flood risk maps or drought indices [2]. The estimated variables can cover different lead times depending on
the characteristics of the extreme event, ranging from short-term prediction to seasonal prediction 
[3].\n\nRecently, deep learning (DL)-based prediction techniques have gained popularity for their ability to 
process large data volumes, capture complex nonlinear relationships, and reduce manual feature engineering. These 
benefits have led to the creation of global models that generalize across various locations, as seen in flood and 
wildfire predictions [4][5].\n\nAlternative probabilistic models, such as surrogate meteorological and hydrological
seasonal re-forecasts, Gaussian processes, and nonlinear dependence measures, have also been proposed for extreme 
event prediction [6]. These models focus on predicting the probability distribution of future variable states, 
rather than providing a single deterministic prediction.\n\nOverall, AI has the potential to significantly improve 
extreme event prediction and understanding, but collaboration across different fields is essential to create 
practical, understandable, and trustworthy AI solutions for analyzing and predicting extreme 
events.\n\nReferences:\n[1] AI for Extreme Event Modeling and Understanding: Methodologies and Challenges, page 
4\n[2] AI for Extreme Event Modeling and Understanding: Methodologies and Challenges, page 4\n[3] AI for Extreme 
Event Modeling and Understanding: Methodologies and Challenges, page 4\n[4] AI for Extreme Event Modeling and 
Understanding: Methodologies and Challenges, page 5\n[5] AI for Extreme Event Modeling and Understanding: 
Methodologies and Challenges, page 5\n[6] AI for Extreme Event Modeling and Understanding: Methodologies and 
Challenges, page 6\n[Quote from Document] User previously responded with tell me something about prediction\n[Quote
from Document] User previously responded with Tell me about events prediction!\n[Quote from Document] Agent 
previously responded with Prediction is the process of estimating the likelihood or outcome of future events using 
various techniques and models. In the context of extreme events, such as droughts, heatwaves, floods, and 
wildfires, prediction is crucial for improving disaster response and communication.\n\nThere are different 
approaches to extreme event prediction. One approach is to model the event as a video prediction task using 
satellite imagery and climate variables. For instance, the vegetation response under extreme drought conditions can
be modeled in this way (AI for Extreme Event Modeling and Understanding: Methodologies and Challenges, page 4). 
Another common approach is to estimate an indicator that defines the extreme event, such as flood risk maps or 
drought indices (AI for Extreme Event Modeling and Understanding: Methodologies and Challenges, page 
4).\n\nRecently, deep learning (DL)-based prediction techniques have gained popularity for their ability to process
large data volumes, capture complex nonlinear relationships, and reduce manual feature engineering (AI for Extreme 

ChatPromptValue(
    messages=[
        SystemMessage(
            content="You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: which are is not still being studied to have a great weather forecasting\n\n From this, we have 
retrieved the following potentially-useful info:  Conversation History Retrieval:\n[Quote from Document] Agent 
previously responded with Sure, I'd be happy to help you understand event prediction, particularly in the context 
of extreme events.\n\nEvent prediction involves using various techniques to estimate the likelihood and potential 
impact of future events. In the context of extreme events, such as droughts, heatwaves, floods, and wildfires, 
event prediction is crucial for improving disaster response and communication.\n\nOne approach to extreme event 
prediction is to model the event as a video prediction task, using satellite imagery and climate variables. For 
example, the vegetation response under extreme drought conditions can be modeled in this way [1]. Another common 
approach is to estimate an indicator that defines the extreme event, such as flood risk maps or drought indices 
[2]. The estimated variables can cover different lead times depending on the characteristics of the extreme event, 
ranging from short-term prediction to seasonal prediction [3].\n\nRecently, deep learning (DL)-based prediction 
techniques have gained popularity for their ability to process large data volumes, capture complex nonlinear 
relationships, and reduce manual feature engineering. These benefits have led to the creation of global models that
generalize across various locations, as seen in flood and wildfire predictions [4][5].\n\nAlternative probabilistic
models, such as surrogate meteorological and hydrological seasonal re-forecasts, Gaussian processes, and nonlinear 
dependence measures, have also been proposed for extreme event prediction [6]. These models focus on predicting the
probability distribution of future variable states, rather than providing a single deterministic 
prediction.\n\nOverall, AI has the potential to significantly improve extreme event prediction and understanding, 
but collaboration across different fields is essential to create practical, understandable, and trustworthy AI 
solutions for analyzing and predicting extreme events.\n\nReferences:\n[1] AI for Extreme Event Modeling and 
Understanding: Methodologies and Challenges, page 4\n[2] AI for Extreme Event Modeling and Understanding: 
Methodologies and Challenges, page 4\n[3] AI for Extreme Event Modeling and Understanding: Methodologies and 
Challenges, page 4\n[4] AI for Extreme Event Modeling and Understanding: Methodologies and Challenges, page 5\n[5] 
AI for Extreme Event Modeling and Understanding: Methodologies and Challenges, page 5\n[6] AI for Extreme Event 
Modeling and Understanding: Methodologies and Challenges, page 6\n[Quote from Document] User previously responded 
with tell me something about prediction\n[Quote from Document] User previously responded with Tell me about events 
prediction!\n[Quote from Document] Agent previously responded with Prediction is the process of estimating the 
likelihood or outcome of future events using various techniques and models. In the context of extreme events, such 
as droughts, heatwaves, floods, and wildfires, prediction is crucial for improving disaster response and 
communication.\n\nThere are different approaches to extreme event prediction. One approach is to model the event as
a video prediction task using satellite imagery and climate variables. For instance, the vegetation response under 
extreme drought conditions can be modeled in this way (AI for Extreme Event Modeling and Understanding: 
Methodologies and Challenges, page 4). Another common approach is to estimate an indicator that defines the extreme
event, such as flood risk maps or drought indices (AI for Extreme Event Modeling and Understanding: Methodologi

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a791f72e9b06fff7a1.gradio.live
Closing server running on port: 7860


<br>

----

<br>

## Saving Your Index For Evaluation


In [ ]:
## Save and compress your index
docstore.save_local("docstore_index")
!tar czvf docstore_index.tgz docstore_index

!rm -rf docstore_index

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss


If everything was properly saved, the following line can be invoked to pull the index from the compressed `tgz` file (assuming the pip requirements are installed).

In [ ]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

# embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
!tar xzvf docstore_index.tgz
new_db = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = new_db.similarity_search("Testing the index")
print(docs[0].page_content[:1000])

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss
. To demonstrate, we build an index using the DrQA [5]\nWikipedia dump from December 2016 and compare outputs from RAG using this index to the newer\nindex from our main results (December 2018). We prepare a list of 82 world leaders who had changed\n7\nTable 4: Human assessments for the Jeopardy\nQuestion Generation Task.\nFactuality\nSpeci\ufb01city\nBART better\n7.1%\n16.8%\nRAG better\n42.7%\n37.4%\nBoth good\n11.7%\n11.8%\nBoth poor\n17.7%\n6.9%\nNo majority\n20.8%\n20.1%\nTable 5: Ratio of distinct to total tri-grams for\ngeneration tasks.\nMSMARCO\nJeopardy QGen\nGold\n89.6%\n90.0%\nBART\n70.7%\n32.4%\nRAG-Token\n77.8%\n46.8%\nRAG-Seq.\n83.5%\n53.8%\nTable 6: Ablations on the dev set. As FEVER is a classi\ufb01cation task, both RAG models are equivalent.\nModel\nNQ\nTQA\nWQ\nCT\nJeopardy-QGen\nMSMarco\nFVR-3\nFVR-2\nExact Match\nB-1\nQB-1\nR-L\nB-1\nLabel Accuracy\nRAG-Token-BM25\n29.7\n41.5\n32.1\n33.1\n17.5\n22